In [1]:
import os
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
from scipy.misc import imresize
from scipy import ndimage

%matplotlib inline

import random
import csv
import re
import pickle
import json
from tqdm import tqdm

DATADIR = os.path.expanduser('~/data/kaggle/ultrasound-nerve-segmentation')
PROCDIR = os.path.expanduser('~/ml/kaggle/ultrasound-nerve-segmentation/processed')
MODELDIR = os.path.expanduser('~/ml/kaggle/ultrasound-nerve-segmentation/models')

In [2]:
with open(os.path.join(PROCDIR, 'train_files_map.pkl'), 'rb') as f:
    train_files_map = pickle.load(f)
with open(os.path.join(PROCDIR, 'test_files_map.pkl'), 'rb') as f:
    test_files_map = pickle.load(f)

## create dataset

In [21]:
orig_img_rows, orig_img_cols = 420, 580
img_rows, img_cols = 96, 128
img_channels = 1

# 90/10 train/val split, by subject
data_masks_train = []
data_masks_val = []

subjects = set([f['subject'] for f in train_files_map.values()])
subjects_val = np.random.choice(list(subjects), int(len(subjects) * 0.1), replace=False)

for file_info in tqdm(train_files_map.values()):
    subject, img, folder, img_file, mask_file = (file_info['subject'], 
                                                 file_info['img'], 
                                                 file_info['folder'], 
                                                 file_info['img_file'], 
                                                 file_info['mask_file'])
    mask_filepath = os.path.join(folder, mask_file)
    mask = io.imread(mask_filepath) / 255.0
    
    if np.sum(mask) == 0:
        continue
        
    mask_resized = imresize(mask, size=(img_rows, img_cols), interp='bilinear').astype(np.bool)
    if subject in subjects_val:
        data_masks_val.append(mask_resized)
    else:
        data_masks_train.append(mask_resized)

data_masks_train = np.array(data_masks_train, dtype=np.bool)
data_masks_val = np.array(data_masks_val, dtype=np.bool)

print('mask shapes:', data_masks_train.shape, data_masks_val.shape)

  6%|▌         | 325/5635 [00:00<00:13, 394.31it/s]/home/leon/install/miniconda3/lib/python3.5/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
100%|██████████| 5635/5635 [00:11<00:00, 486.91it/s]

mask shapes: (2141, 96, 128) (182, 96, 128)


## augmentation

In [19]:
multiplier = 2

data_masks_train_augment = []
for i in tqdm(range(data_masks_train.shape[0])):
    data_masks_train_augment.append(data_masks_train[i])
    for j in range(multiplier):
        y_shift = random.randint(int(data_masks_train.shape[1] * -0.125), 
                                 int(data_masks_train.shape[1] * 0.125))
        x_shift = random.randint(int(data_masks_train.shape[2] * -0.125), 
                                 int(data_masks_train.shape[2] * 0.125))
        mask_aug = ndimage.interpolation.shift(data_masks_train[i], (y_shift, x_shift), order=0)
        data_masks_train_augment.append(mask_aug)

data_masks_train = np.array(data_masks_train_augment, dtype=np.bool)

print('mask shapes:', data_masks_train.shape, data_masks_val.shape)

100%|██████████| 2139/2139 [00:00<00:00, 2939.89it/s]

mask shapes: (6417, 96, 128) (184, 96, 128)


## save

In [22]:
with open(os.path.join(PROCDIR, 'vae_masks_train_val_16.pkl'), 'wb') as f:
    pickle.dump((data_masks_train[0:2093,:,:], data_masks_val), f, protocol=4)